In [ ]:
import requests, pymongo, json, time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup

![](http://www.listchallenges.com/f/lists/d7aacdae-74bd-42ff-b397-b73905b5867b.jpg)

## Introduction

Our goal is to scape as many New York Times articles from their api as possible. The web scraper below is desgined to open a mongo client, where all the downloaded articles will be stored and make api calls. This web scraper will run indefinately until the user termintes the code's calls. 

## New York Times Web Scraper

In [1]:
import pymongo

client = pymongo.MongoClient()

In [2]:
import requests
import time
import json
from bs4 import BeautifulSoup

def init_mongo_client():
    # Initiate Mongo client
    client = pymongo.MongoClient()

    # Access database created for these articles
    db = client.nyt_articles
    
    # Access collection created for these articles
    coll = db.NYT_Articles

    # Access articles collection in db and return collection pointer.
    return db.articles


def call_api(url, payload, p=0):
    # Add the 'page' parameter to the payload.
    payload['page'] = p

    # Get the requested url. Error handling for bad requests should be done in
    # the calling function.
    return requests.get(url, params=payload)


def get_response(r):
    # Use json.loads to read the response text
    raw = json.loads(r.text)

    # Return the meta (hits, etc.) and docs (containing urls'n'stuff) back
    return raw['response']['meta'], raw['response']['docs']


def get_soup(url):
    # Header to be passed in to NYT when scraping article text.
    agent  = 'DataWrangling/1.1 (http://zipfianacademy.com; '
    agent += 'class@zipfianacademy.com)'
    headers = {'user_agent': agent}

    # Wrap in a try-except to prevent a maxTry connection error from erroring
    # out the program. Return None if there are any issues.
    try:
        r = requests.get(url, headers=headers)
    except:
        return None

    # Just in case there was a normal error returned. Pass back None.
    if r.status_code != 200: return None

    # Otherwise return a soupified object containing the url text encoded in
    # utf-8. Will toss back errors on some pages without the encoding in place.
    return BeautifulSoup(r.text.encode('utf-8'))


def get_body_text(docs):

    # Grab the url from each document, if it exists, then scrape each url for
    # its body text. If we get any errors along the way, continue on to the
    # next document / url to be scraped.
    result = []
    for d in docs:

        # Make a copy of the doc's dictionary
        doc = d.copy()

        # If there's no url (not sure why this happens sometimes) then ditch it
        if not doc['web_url']:
            continue

        # Scrape the doc's url, return a soup object with the url's text.
        soup = get_soup(doc['web_url'])
        if not soup:
            continue

        # Find all of the paragraphs with the correct class.
        # This class tag is specific to NYT articles.
        body = soup.find_all('p', class_= "story-body-text story-content")
        if not body:
            continue

        # Join the resulting body paragraphs' text (returned in a list).
        doc['body'] = '\n'.join([x.get_text() for x in body])

        print (doc['web_url'])
        result.append(doc)

    return result


def remove_previously_scraped(coll, docs):
    # Check to see if the mongo collection already contains the docs returned
    # from NYT. Return back a list of the ones that aren't in the collection to
    # be scraped.
    new_docs = []
    for doc in docs:
        # Check fo the document id in mongo. If it finds none, append to
        # new_docs
        cursor = articles.find({'_id': doc['_id']}).limit(1)
        if not cursor.count() > 0:
            new_docs.append(doc)

    if new_docs == []:
        return None

    return new_docs


def get_end_date(dt):
    # String-ify the datetime object to YYYMMDD, which the NYT likes.
    yr   = str(dt.year)
    mon = '0' * (2 - len(str(dt.month))) + str(dt.month)
    day = '0' * (2 - len(str(dt.day))) + str(dt.day)
    return yr + mon + day


def scrape_articles(coll, last_date):
    print(API_KEY)
    page = 0
    while page < 200:
        print(get_end_date(last_date))
        # Request all of the newest articles matching the search term
        payload  = {'sort': 'newest',
                    'end_date': get_end_date(last_date),
                    'api-key': "8c8263e4c6c347d685349a55607e82b2"
                   }

        # Call the API with BaseURL + params and page
        r = call_api(NYT_URL, payload, page)

        # Increment the page before we encounter any potential errors
        page += 1

        # Check to see if the metadata didn't come back. USUALLY happens if
        # page > 100. When it does, reset the whole thing, roll the date back
        # one day, sleep for a couple seconds, then keep going.
        if r.status_code != 200:
            page = 0
            last_date += relativedelta(days=-1)
            print ('End Date:', get_end_date(last_date))
            print (r.status_code )
            time.sleep(2)
            continue
            
        # Get the meta data & documents from the request
        meta, docs = get_response(r)

        # Check if docs are already in the database
        new_docs = remove_previously_scraped(coll, docs)
        if not new_docs: continue

        # Grab only the docs that have these tags
        docs_with_body = get_body_text(new_docs)

        for doc in docs_with_body:
            try:
                # Insert each doc into Mongo
                coll.insert_one(doc)
            except:
                # If there's any error writing it in the db, just move along.
                print("couldn't write to DB")
                continue

In [3]:
# Url for NYT dev api
NYT_URL = 'http://api.nytimes.com/svc/search/v2/articlesearch.json'
api_key_path = "NYT_API_Key.txt"

In [4]:
with open(api_key_path, 'r') as handle:
    API_KEY = handle.read()
print(API_KEY)

a5b965c8a15c47539bbbbb391ff03157



In [5]:
# Initialize db & collection
articles = init_mongo_client()

In [6]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# Set the initial end date (scraper starts at this date and moves back in
# time sequentially)
last_date = datetime.now() + relativedelta(days=-590)

In [ ]:
# Pass the database collection and initial end date into main function
scrape_articles(articles, last_date)

a5b965c8a15c47539bbbbb391ff03157

20160412


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html.parser")

  markup_type=markup_type))


https://www.nytimes.com/2016/04/12/opinion/what-states-can-do-on-birth-control.html
https://www.nytimes.com/2016/04/12/opinion/a-home-but-not-home.html
https://www.nytimes.com/2016/04/12/nytnow/your-tuesday-briefing-dilma-rousseff-david-cameron-zika-virus.html
https://www.nytimes.com/2016/04/12/pageoneplus/corrections-april-12-2016.html
https://www.nytimes.com/2016/04/12/t-magazine/mens-swimwear-lacma.html
20160412
https://www.nytimes.com/2016/04/12/arts/television/what-to-watch-tuesday.html
https://www.nytimes.com/2016/04/12/business/alan-dershowitz-and-2-other-lawyers-settle-suit-and-counter-claim.html
https://www.nytimes.com/2016/04/12/books/review/beverly-cleary-turns-100.html
https://www.nytimes.com/2016/04/12/theater/small-mouth-sounds-returns-to-new-york.html
https://www.nytimes.com/2016/04/12/sports/soccer/wayne-rooney-begins-his-comeback.html
20160412
https://www.nytimes.com/2016/04/12/t-magazine/courreges-sebastien-meyer-arnaud-vaillant-visit-pau-workshop.html
https://www.nyt